In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
import torch.nn.functional as F
from torchsummary import summary
from tqdm import tqdm
use_gpu = torch.cuda.is_available()
torch.cuda.set_device(2)


## DATASETS & DATALAODER

In [2]:
data_dir = "/home/jovyan/EJ/causality/Caltech-Birds-Classification/train/"
trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
image_datasets = datasets.ImageFolder(data_dir,trans)
dataloders = torch.utils.data.DataLoader(image_datasets, batch_size=8,
                                             shuffle=True)
dataset_sizes = len(image_datasets)


## EVAL MODEL

In [3]:
vgg_eval = models.vgg16(pretrained=True).eval()
if use_gpu:             
    vgg_eval = vgg_eval.cuda()       
vgg_eval = vgg_eval.float()


## INTERVENE MODEL

In [4]:
vgg_test = models.vgg16(pretrained=True)
if use_gpu:                                 # if gpu is available then use it
    vgg_test = vgg_test.cuda()       
vgg_test = vgg_test.float()
summary(vgg_test,(3,224,224))
# vgg_test

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

## MODIFY MODEL

In [5]:
freeze_layer = 9
f_model = list(vgg_test.features.children())[:freeze_layer]
f_model = nn.Sequential(*f_model)
r_model = list(vgg_test.features.children())[freeze_layer:len(list(vgg_test.features.children()))]
r_model = nn.Sequential(*r_model)
cls_model = nn.Sequential(*list(vgg_test.classifier.children()))
r_freeze = list(vgg_test.features.children())[freeze_layer:len(list(vgg_test.features.children()))]
r_freeze = nn.Sequential(*r_freeze)
cls_freeze = nn.Sequential(*list(vgg_test.classifier.children()))

for param in r_freeze.parameters():
    param.requires_grad = False
for param in cls_freeze.parameters():
    param.requires_grad = False

In [6]:
ae1 = nn.Conv2d(in_channels=128,out_channels=32,kernel_size=(3,3),padding=(1, 1))
hd1 = nn.Conv2d(in_channels=32,out_channels=16,kernel_size=(3,3),padding=(1, 1))
hd2 = nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(3,3),padding=(1, 1))
ae2 = nn.Conv2d(in_channels=32,out_channels=128,kernel_size=(3,3),padding=(1, 1))

In [7]:
class vgg_auto(nn.Module):
    def __init__(self):
        super(vgg_auto, self).__init__()
        self.f_model = f_model
        self.ae1 = ae1
        self.hd1 = hd1
        self.hd2 = hd2
        self.ae1 = ae1
        self.ae2 = ae2
        self.r_model = r_model
        self.r_freeze = r_freeze
        self.r_model = r_model
        self.cls_model = cls_model
        self.cls_freeze = cls_freeze
        self.zero_out = np.random.randint(0,16)
        self.prob_zero = np.random.uniform(0,1,1)
        
    def forward_interpret(self,x):
        x = hd1(x)
        x = hd2(x)
        return x
    
    def forward_shallow(self,x):
        x = self.ae1(x)
        y1 = x.clone()
        x = self.forward_interpret(x)
        y2 = x.clone()
        x = self.ae2(x)
        return x, y1,y2
    
    def forward(self, x):
        x = self.f_model(x)
        x2 = x.clone()
        y, y1,y2 = self.forward_shallow(x)
        y3 = y.clone()
        x = self.r_model(x)
        x = x.view(x.size(0), -1)
        x = self.cls_model(x)
        y = r_freeze(y)
        y = y.view(y.size(0), -1)
        y = cls_freeze(y)
        return x, x2, y, y1, y2, y3


In [8]:
def tvloss(yhat, y):
    bsize, chan, height, width = y.size()
    errors = []
    dy = torch.abs(y[:,:,1:,:] - y[:,:,:-1,:])
    dyhat = torch.abs(yhat[:,:,1:,:] - yhat[:,:,:-1,:])
    error = torch.norm(dy - dyhat, 1)
    return error / height

## TRAIN FUNC

In [9]:
save_path = "causal_fine_overlap.pth"
def train_causal_model(c_model, optimizer, num_epochs=10):
    best_model_wts = c_model.state_dict()
    for epoch in range(num_epochs):
        c_model.train(True)  # Set model to training mode
        
        running_loss = 0.0
        test_loss = 10

        for data in dataloders:
            inputs,labels = data
            if use_gpu:
                inputs = Variable(inputs.cuda())
            else:
                inputs = Variable(inputs)

            optimizer.zero_grad()
            
            x,x2,y,y1,y2,y3 = c_model(inputs)
#             loss_interpret = nn.CrossEntropyLoss()(y1,y2.detach().long())
            loss_interpret = tvloss(y1,y2)
            loss_ae = nn.L1Loss()(x2,y3.detach())
            loss_kl = nn.KLDivLoss()(F.log_softmax(y,-1), F.softmax(x,-1).detach())

            loss = 5*loss_interpret + 2*loss_ae + 3*loss_interpret
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        epoch_loss = running_loss / dataset_sizes
#         if epoch % 100 == 0:
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('Loss: {:.10f} '.format(epoch_loss))

        # deep copy the model
        if test_loss > epoch_loss:
            test_loss = epoch_loss
            best_model_wts = c_model.state_dict()
            state = {'model':c_model.state_dict(),'optim':optimizer.state_dict()}
            torch.save(state,save_path)

    # load best model weights
    c_model.load_state_dict(best_model_wts)
    return c_model

In [10]:
causal_model = vgg_auto().cuda()
optimizer_c = optim.Adam(filter(lambda p: p.requires_grad,causal_model.parameters()), lr=0.001)
# for name, param in causal_model.named_parameters():
#     if "r_model" in name:
#         param.requires_grad = True
#     if param.requires_grad:
#         print(name)

In [11]:
train_causal_model(causal_model,optimizer_c,num_epochs=3)

Epoch 0/2
Loss: 41.7687948142 
Epoch 1/2
Loss: 0.0435308144 
Epoch 2/2
Loss: 0.0392419569 


vgg_auto(
  (f_model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
  )
  (ae1): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (hd1): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (hd2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (ae2): Conv2d(32, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (r_model): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1,

# FOOL BOX

In [12]:
from PIL import Image
import cv2

image_path = "train/n01580077/63.jpg"

img_fool = Image.open(image_path)
trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
img_fool = trans(img_fool)
# img_fool.unsqueeze_(0)
img_fool.cuda()
print("img to cuda")
# img_fool = img_fool.cpu().numpy()
print(torch.max(img_fool))

import foolbox
from foolbox.models import PyTorchModel
vgg_fool = models.vgg16(pretrained=True).eval()
vgg_fool.cuda()

fool_model = foolbox.models.PyTorchModel(vgg_fool, bounds=(-3,3), num_classes=1000)

img_fool = img_fool.cpu().numpy()
print(img_fool.shape)
print(np.argmax(fool_model.predictions(img_fool)))
label = np.argmax(fool_model.predictions(img_fool))

# apply attack on source image
attack = foolbox.attacks.FGSM(fool_model)
adversarial = attack(img_fool, label)
print('adversarial class', np.argmax(fool_model.predictions(adversarial)))


img to cuda
tensor(2.6400)
(3, 224, 224)
17
adversarial class 89


# CAUSAL EFFECT

In [13]:
from PIL import Image
import cv2
import matplotlib.cm as cm
def computingECE(causal_model,img):
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    trans = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
    img = trans(img)
    img.unsqueeze_(0)
    img = img.cuda()
    outputs = causal_model(img)
    score = F.softmax(outputs[0],1).view(-1)
    print(torch.max(score,0))
#     print(score)
    
    causal_score = F.softmax(outputs[2],1).view(-1)
    print(torch.max(causal_score,0))
    causal_effect = causal_score -score
#     print(causal_effect)
    ece = torch.dot(causal_effect,score)
    return ece

In [14]:
image_path = "train/n01531178/62.jpg"
ece = computingECE(causal_model,adversarial)

(tensor(1.00000e-02 *
       1.0051, device='cuda:2'), tensor(812, device='cuda:2'))
(tensor(1.00000e-02 *
       1.3776, device='cuda:2'), tensor(892, device='cuda:2'))


In [15]:
ece

tensor(1.00000e-04 *
       -2.8862, device='cuda:2')